# Naive Bayes Classifier

In [21]:
import nltk
from nltk.corpus import movie_reviews
nltk.download("movie_reviews")

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [22]:
from collections import defaultdict, Counter
import math
import random

# X: words / Y: polarity
train_X, train_Y = [], []
test_X, test_Y = [], []

random.seed(0)
for polarity in movie_reviews.categories():
    for fid in movie_reviews.fileids(polarity):
        # 1/5 for test; 4/5 for train
        if random.randrange(5) == 0:
            test_X.append([w for w in movie_reviews.words(fid)])
            test_Y.append(polarity)
        else:
            train_X.append([w for w in movie_reviews.words(fid)])
            train_Y.append(polarity)

print(train_X[0], train_Y[0])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch', 'the', 'movie', 'and', '"', 'sorta', '"', 'find', 'out', '.', '.', '.', 'critique', ':', 'a', 'mind', '-', 'fuck', 'movie', 'for', 'the', 'teen', 'generation', 'that', 'touches', 'on', 'a', 'very', 'cool', 'idea', ',', 'but', 'presents', 'it', 'in', 'a', 'very', 'bad', 'package', '.', 'which', 'is', 'what', 'makes', 'this', 'review', 'an', 'even', 'harder', 'one', 'to', 'write', ',', 'since', 'i', 'generally', 'applaud', 'films', 'which', 'attempt', 'to', 'break', 'the', 'mold', ',', 'mess', 'with', 'your', 'head', 'and', 'such', '(', 'lost', 'highway', '&', 'memento', ')', ',', 'but', 'there', 'are', 'good', 'and', 'ba

## Model Construction

$\bar{y} = \text{arg}\max_{y \in \mathbf{y}} P(y|x) = \text{arg}\max_{y \in \mathbf{y}} P(y) \prod_{i=1}^n \frac{P(x_i|y)}{P(x_i)} = \text{arg}\max_{y \in \mathbf{y}} P(y) \prod_{i=1}^n P(x_i|y)$

$P(x_i|y)=\frac{C(x_i, y) + k}{C(y) + |\mathbf{y}| \times k}$

$\bar{y} = \textrm{arg} \max_{y \in \mathbf{y}} \log P(y) + \sum_{i=1}^n \log \frac{C(x_i, y) + k}{C(y) + k|\mathbf{y}|}$

     

In [350]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import numpy as np
nltk.download('wordnet')
nltk.download('stopwords')

class NaiveBayesClassifier:
    def __init__(self, k=0.1):
        self.k = k
        self.features = set()
        self.class_feature_counts = defaultdict(Counter)
        self.class_counts = Counter()
        self.total = 0
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.vectorizer = CountVectorizer(max_df=0.65, min_df=10)
        self.selector = SelectKBest(mutual_info_classif, k=2000)

    def train(self, train_X, train_Y):
        word_counts = self.vectorizer.fit_transform([' '.join(tokens) for tokens in train_X])
        self.selector.fit(word_counts, train_Y)
        feature_names = np.array(self.vectorizer.get_feature_names_out())[self.selector.get_support()]

        for tokens, label in zip(train_X, train_Y):
            self.class_counts[label] += 1
            self.total += 1
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]
            for token in set(tokens):
                if token in feature_names:
                    self.features.add(token)
                    self.class_feature_counts[label][token] += 1

    def probabilities(self, token):
        probs = {}
        for cls, cls_cnt in self.class_counts.items():
            probs[cls] = (self.class_feature_counts[cls][token] + self.k) / (cls_cnt + len(self.class_counts) * self.k)
        return probs

    def predict(self, tokens):
        tokens = set([self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words])
        log_probs = Counter()
        for cls, cls_cnt in self.class_counts.items():
            log_probs[cls] = math.log(cls_cnt / self.total)
        for token in self.features:
            probs = self.probabilities(token)
            if token in tokens:
                for cls, prob in probs.items():
                    log_probs[cls] += math.log(prob)
            else:
                for cls, prob in probs.items():
                    log_probs[cls] += math.log(1.0 - prob)
        # Return the argmax of log_probs and all log_probs
        return max(log_probs, key=log_probs.get), log_probs

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Using the Model

In [351]:
model = NaiveBayesClassifier()
model.train(train_X, train_Y)

In [352]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Taken from https://www.imdb.com/review/rw0990793/?ref_=tt_urv
review = """A whimsical, often spectacular view of a future in which advances in technology dominate the world. It is well shot and although slow-moving it is intense and enjoyable throughout. The featuring of classical music to establish atmosphere works brilliantly; it provides a feeling of awe, mystery and intrigue  the same aura that Walt Disney worked in creating 'Fantasia'. The special effects, both sound and visual, are still spellbinding by the standards of today's technology. Aside from the technical pluses of the film, it stands strong as it is one of not many films out there that has something important to say about humankind, and where the human race is heading in terms of our increasing reliance on machines and our unquenchable thirst to discover. Despite an ending that is hard to understand, it is even harder to overlook this film a true cinema classic."""

model.predict(word_tokenize(review))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('pos', Counter({'neg': -195.86971202641826, 'pos': -192.72106891678405}))

In [353]:
correct, total = 0, 0

for x, y in zip(test_X, test_Y):
    prediction, _ = model.predict(x)
    if prediction == y:
        correct += 1
    total += 1

print("%d / %d = %g" % (correct, total, correct / total))

362 / 422 = 0.85782


## Exploring important features

In [354]:
def prob_class_given_feature(feature, cls, model):
    probs = model.probabilities(feature)
    return probs[cls] / sum(probs.values())

print(sorted(model.features, key=lambda t: prob_class_given_feature(t, "pos", model), reverse=True)[:30])
print(sorted(model.features, key=lambda t: prob_class_given_feature(t, "neg", model), reverse=True)[:30])

['thematic', 'dread', 'astounding', 'reminder', 'kenobi', 'turturro', 'naval', 'en', 'seamless', 'fascination', 'denial', 'lovingly', 'keen', 'masterfully', 'vocal', 'maintains', 'strongest', 'effortlessly', 'fable', 'concentration', 'avoids', 'guardian', 'darker', 'affecting', 'accessible', 'outstanding', 'hatred', 'ideology', 'annual', 'rousing']
['hudson', 'sans', '3000', 'illogical', 'overwrought', 'don', 'plant', 'schumacher', 'uh', 'ugh', 'roberts', 'henstridge', 'insulting', 'welles', 'excruciatingly', 'unimaginative', 'fairness', 'ludicrous', 'terri', 'furniture', 'predator', 'beard', 'silverstone', 'wasting', 'coyote', 'justin', 'turkey', 'insipid', 'suvari', 'tripe']
